In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import pickle


In [2]:
loaded=CountVectorizer(decode_error='replace',vocabulary=pickle.load(open('word_feats.pkl','rb')))

In [3]:
user_df = pd.DataFrame(columns = ['comment_text'])
user_df


Empty DataFrame
Columns: [comment_text]
Index: []

In [4]:
inp_comment =  str(input("enter a comment : "))
inp_comment


enter a comment : this painting is good


'this painting is good'

In [5]:
new_row = {'comment_text':inp_comment}
#append a new row to the dataframe formed by user inputs
user_df = user_df.append(new_row,ignore_index = True)

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [7]:
user_df['comment_text'] = user_df['comment_text'].map(lambda com : clean_text(com))
user_df

comment_text
0  this painting is good

In [8]:
user_text = user_df['comment_text']

In [9]:
user_features = loaded.transform(user_text)
print(user_features)


  (0, 123753)	1
  (0, 212149)	1


In [10]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
lst= []
mapper = {}
for label in cols_target:
    filename = str(label+'_model.sav')
    filename
    model = pickle.load(open(filename, 'rb'))
    print('... Processing {}'.format(label))
    user_y_prob = model.predict_proba(user_features)[:,1]
    print(label,":",user_y_prob[0])
    lst.append([label,user_y_prob])
    #submission_binary[label] = test_y_prob
print(lst)

... Processing obscene
obscene : 0.01968485945454813
... Processing insult
insult : 0.027084203993887373
... Processing toxic
toxic : 0.033520172624573304
... Processing severe_toxic
severe_toxic : 0.012791298341818512
... Processing identity_hate
identity_hate : 0.013851485067844867
... Processing threat
threat : 0.002288283974443752
[['obscene', array([0.01968486])], ['insult', array([0.0270842])], ['toxic', array([0.03352017])], ['severe_toxic', array([0.0127913])], ['identity_hate', array([0.01385149])], ['threat', array([0.00228828])]]


In [13]:
final=[]
for i in lst:
    if i[1]>0.5:
        final.append(i[0])

if not len(final):
    text = "The comment is clean"
else:
    text=final

In [14]:
print(text)


The comment is clean
